In [ ]:
import pickle

import lightgbm
from lightgbm import LGBMRegressor
from sklearn.linear_model import LogisticRegression

from lightgbm import LGBMClassifier
import csv
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import KFold, RandomizedSearchCV, GridSearchCV
import xgboost as xgb
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor




In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/player-20-clean/player_20_clean.csv', index_col = False)
val_nl = df[df['value_eur']==0].index.tolist()
df.drop(df.index[val_nl],inplace= True)
y = df['value_eur'].values
df.drop(['value_eur'],axis = 1 , inplace = True)
x = df.values

In [ ]:
df = pd.read_csv('/kaggle/input/my-fifa-19/final_data_v3.csv', index_col = False)
# pos_nl = df[df['Value']==0].index.tolist()
# print ('Number of null positions: ', len(pos_nl))
# df.drop(df.index[pos_nl],inplace= True)
pos_nl = df[df['LS']==0].index.tolist()
val_nl = df[df['Value']==0].index.tolist()
total_nl = list(set(pos_nl + val_nl))
print ('Number of Goalkeepers ', len(pos_nl))
df.drop(df.index[total_nl],inplace= True)

In [ ]:
y = df['Value'].values
df.drop(['Value', 'Jersey Number', 'Joined','Contract Valid Until', 'Preferred Foot'], inplace=True, axis=1)
#df.drop(['Value'], axis = 1 , inplace = True)
x = df.values

In [ ]:
df  = pd.read_csv('/kaggle/input/data-42/data_42.csv', index_col = False)
val_nl = df[df['Value']==0].index.tolist()
print ('Number of Goalkeepers ', len(pos_nl))
df.drop(df.index[val_nl],inplace= True)

In [ ]:
y = df['Value'].values
df.drop(['Value'], axis = 1 , inplace = True)
x = df.values

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, Columns):
        self.Columns=Columns
        
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        New_X=X.copy()
        New_X=New_X[self.Columns].copy()
        return New_X

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

pipeline=Pipeline([
    ('custom_tr', CustomTransformer(Skill_cols)),
    ('imputer',SimpleImputer(strategy='median')),
    ('std_scaler',StandardScaler())
])

In [ ]:
x=pipeline.fit_transform(df)
y=df['value_eur'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.252, random_state=2)

print (X_train.shape, X_test.shape)

In [ ]:
def k_fold(X, Y, clf):
    #  shuffle dataset randomly , split the dataset into 10 groups
    kf = KFold(n_splits=10, random_state=1, shuffle=True)

    rmses= []
    rmses_lg= []
    #Y=  Y/100000
    yp = []
    yt = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        # data normalization with sklearn
        X_train = MinMaxScaler().fit_transform(X_train)
        #Y_train = MinMaxScaler().fit_transform(Y_train.reshape(-1,1))
        Y_train = np.log(Y_train)
        
        #print (X_train)
        clf.fit(X_train, Y_train.ravel())
        
        X_test= MinMaxScaler().fit_transform(X_test)
        #norm_test = MinMaxScaler().fit(Y_test.reshape(-1, 1))
        #Y_test = norm_test.transform(Y_test.reshape(-1, 1))
        Y_pred = clf.predict(X_test)
        Y_test_lg = np.log (Y_test)
        #yp = np.rint(Y_pred)
        #yt = Y_test
        rmse_lg = mean_squared_error(Y_test_lg,Y_pred) ** 0.5
        print('The rmse of prediction is:', rmse_lg)
        rmses_lg.append(rmse_lg)
        
        rmse = mean_squared_error(Y_test,np.exp(Y_pred)) ** 0.5
        print('The rmse of prediction is:', rmse)
        rmses.append(rmse)
    
    average_rmse = np.mean(np.array(rmses))
    average_rmse_lg = np.mean(np.array(rmses_lg))
    #yt = norm_test.inverse_transform(Y_test)
    #yp = norm_test.inverse_transform(Y_pred.reshape(-1,1))
    return clf, average_rmse,average_rmse_lg,Y_pred,  Y_test_lg



In [ ]:
def k_fold_scl(X, Y, clf):
    #  shuffle dataset randomly , split the dataset into 10 groups
    kf = KFold(n_splits=10, random_state=1, shuffle=True)

    rmses= []
    rmses_scl = []
    Y=  Y/1000000
    yp = []
    yt = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        # data normalization with sklearn
        X_train = MinMaxScaler().fit_transform(X_train)
        
        #print (X_train)
        clf.fit(X_train, Y_train.ravel())
        
        X_test= MinMaxScaler().fit_transform(X_test)
        Y_pred = clf.predict(X_test)
        rmse = np.sqrt(mean_squared_error(Y_test,Y_pred))
        rmse_scl = mean_squared_error(Y_test*1000000,Y_pred*10000000) ** 0.5
        print('The rmse of prediction is:', rmse)
        rmses.append(rmse)
        rmses_scl.append(rmse_scl)
        
        
    
    avg_rmse = np.mean(np.array(rmses))
    avg_rmse_scl = np.mean(np.array(rmses_scl))
    return avg_rmse,avg_rmse_scl,  Y_pred,Y_test



In [ ]:
def model_predict(clf, X_test, Y_test):
        X_test= MinMaxScaler().fit_transform(X_test)
        Y_pred = clf.predict(X_test)
        Y_test_lg = np.log (Y_test)

        rmse_lg = mean_squared_error(Y_test_lg,Y_pred) ** 0.5
        print('The rmse of prediction is:', rmse_lg)
        
        rmse = mean_squared_error(Y_test,np.exp(Y_pred)) ** 0.5
        print('The rmse of prediction is:', rmse)
        #rmses.append(rmse)
        return rmse_lg, rmse , Y_pred

Models

In [ ]:
clf_lr = LinearRegression()
clf_lgbm = lightgbm.LGBMRegressor(boosting_type=  'gbdt', learning_rate =  0.1, num_leaves =  31)
clf_xgb = XGBRegressor(booster =  'gbtree', learning_rate =  0.1)
clf_cat = CatBoostRegressor(iterations = 100, learning_rate = 0.1)
clf_rf = RandomForestRegressor(max_features= 10, n_estimators= 30)
model_names = list(['lr', 'rfr', 'lgbmr', 'xgbr', 'catbr'])
models = list([clf_lr,  clf_rf, clf_lgbm , clf_xgb, clf_cat])


In [ ]:
result = []
prd_list = []
for idx, model in enumerate(models) :
    X_train_cp= np.copy(X_train)
    y_train_cp= np.copy(y_train)
    print (model_names[idx])
    fit_model, rmse, rmsel, ypl, ytl = k_fold(X_train,y_train,model)
    trmsel, trmse, yp = model_predict (fit_model, np.copy(X_test), np.copy(y_test))
    result.append(list([model_names[idx]]) + list([rmse, trmse, rmsel, trmsel]))
    prd_list.append(yp)
odf = pd.DataFrame(result, columns = list(['model_name', 'rmse', 'trmse','rmsel','trmsel'] ))
#fdf.append()
odf.to_csv('original_dataset_result.csv', index = False)

All features

In [ ]:
fdf = odf.copy()
fdf

42 features

In [ ]:
fdf_42 = odf.copy()
fdf_42

In [ ]:
prd_list

All Features

Logarithmic Scale

In [ ]:
for idx, prd in enumerate(prd_list):
    # print ()
    ypl = prd
    ytl = np.copy(np.log(y_test))
    plt.rcParams['figure.figsize'] = [20, 5]
    fig, ax = plt.subplots()
    ax.scatter(ytl, ypl)
    ax.plot([ytl.min(), ytl.max()], [ytl.min(), ytl.max()], 'k--', lw=4)
    ax.set_xlabel(model_names[idx] + ' Measured')
    ax.set_ylabel('Predicted')
    plt.show()

Normal Scale

In [ ]:
for idx, prd in enumerate(prd_list):
    # print ()
    ypl = np.exp(prd)
    ytl = np.copy(y_test)
    plt.rcParams['figure.figsize'] = [20, 5]
    fig, ax = plt.subplots()
    ax.scatter(ytl, ypl)
    ax.plot([ytl.min(), ytl.max()], [ytl.min(), ytl.max()], 'k--', lw=4)
    ax.set_xlabel(model_names[idx] + ' Measured')
    ax.set_ylabel('Predicted')
    plt.show()

42 features

Logarithmic scale

In [ ]:
for idx, prd in enumerate(prd_list):
    # print ()
    ypl = prd
    ytl = np.copy(np.log(y_test))
    plt.rcParams['figure.figsize'] = [20, 5]
    fig, ax = plt.subplots()
    ax.scatter(ytl, ypl)
    ax.plot([ytl.min(), ytl.max()], [ytl.min(), ytl.max()], 'k--', lw=4)
    ax.set_xlabel(model_names[idx] + ' Measured')
    ax.set_ylabel('Predicted')
    plt.show()

Normal Scale

In [ ]:
for idx, prd in enumerate(prd_list):
    # print ()
    ypl = np.exp(prd)
    ytl = np.copy(y_test)
    plt.rcParams['figure.figsize'] = [20, 5]
    fig, ax = plt.subplots()
    ax.scatter(ytl, ypl)
    ax.plot([ytl.min(), ytl.max()], [ytl.min(), ytl.max()], 'k--', lw=4)
    ax.set_xlabel(model_names[idx] + ' Measured')
    ax.set_ylabel('Predicted')
    plt.show()
